Sobre los ejemplos realizados en clase de los modelos CNN MobileNet y Efficienent corre Tensorboard para analizar las capas


In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [6]:
# 1. Importar librerías
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import datetime

# 2. Cargar y preprocesar datos
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0
y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)

# 3. Arquitectura estilo VGG
model = Sequential([
    Conv2D(32, (3, 3), activation='relu',
           padding='same', input_shape=(32, 32, 3)),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu', padding='same'),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

# 4. Compilar y entrenar
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

# Log dir para TensorBoard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    write_graph=True
)

history = model.fit(x_train, y_train_cat, epochs=5, batch_size=64, validation_split=0.1, verbose=2,
                    callbacks=[tensorboard_callback])


Epoch 1/5
704/704 - 72s - 102ms/step - accuracy: 0.4485 - loss: 1.5068 - val_accuracy: 0.5970 - val_loss: 1.1402
Epoch 2/5
704/704 - 68s - 97ms/step - accuracy: 0.6286 - loss: 1.0468 - val_accuracy: 0.6914 - val_loss: 0.9003
Epoch 3/5
704/704 - 67s - 95ms/step - accuracy: 0.7020 - loss: 0.8522 - val_accuracy: 0.7410 - val_loss: 0.7632
Epoch 4/5
704/704 - 67s - 95ms/step - accuracy: 0.7461 - loss: 0.7245 - val_accuracy: 0.7648 - val_loss: 0.7002
Epoch 5/5
704/704 - 70s - 100ms/step - accuracy: 0.7819 - loss: 0.6248 - val_accuracy: 0.7690 - val_loss: 0.6852


In [7]:
@tf.function(input_signature=[tf.TensorSpec([None, 32, 32, 3], tf.float32)])
def traced_model(x):
    return model(x)


writer = tf.summary.create_file_writer(log_dir)
with writer.as_default():
    tf.summary.graph(traced_model.get_concrete_function(tf.zeros([1, 32, 32, 3])).graph)

In [8]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 2512), started 1:07:57 ago. (Use '!kill 2512' to kill it.)